# Alert Production Data Quality Report for {{params.instrument}} on {{ params.date }}

In [ ]:
# Parameters.  Set defaults here.

date = "2024-03-29"
instrument = "LATISS"

In [ ]:
import numpy as np

In [ ]:
day_obs = int(date.replace("-", ""))
day_obs

In [ ]:
match instrument:
    case "LATISS":
        sal_index = 2
        n_detector = 1
    case "LSSTComCamSim":
        sal_index = 3
        n_detector = 9
    case _:
        logger.error(f"Unknown instrument {instrument}")

In [ ]:
# https://rtn-045.lsst.io/#colorblind-friendly-plots
plot_filter_colors = {
    "u": "#56b4e9",
    "g": "#008060",
    "r": "#ff4000",
    "i": "#850000",
    "z": "#6600cc",
    "y": "#000000",
}

1.3.14 Level 1 Data Quality Report Definition

ID: DMS-REQ-0097 (Priority: 1a)

Specification: The DMS shall produce a Level 1 Data Quality Report that contains indicators
of data quality that result from running the DMS pipelines, including at least: Photometric
zero point vs. time for each utilized filter; Sky brightness vs. time for each utilized filter; seeing
vs. time for each utilized filter; PSF parameters vs. time for each utilized filter; detection
efficiency for point sources vs. mag for each utilized filter.

Discussion: The seeing report is intended as a broad-brush measure of image quality. The
PSF parameters provide more detail, as they include asymmetries and field location
dependence

In [ ]:
import matplotlib.pyplot as plt
from lsst_efd_client import EfdClient

As of 8 Feb 24, `lsst.prompt` is only available in usdfdev while we await authentication. 

In [ ]:
client = EfdClient("usdfdev_efd", db_name="lsst.prompt")

In [ ]:
await client.get_topics()

In [ ]:
query = (
    f'''SELECT * FROM "lsst.prompt.numDiaSourcesAll"'''  # where day_obs = {day_obs} '''
)
print(query)
df = await client.influx_client.query(query)
df

In [ ]:
help(client.get_timeseries)

In [ ]:
np.sum(df.instrument == instrument)

In [ ]:
query = f"""SELECT * FROM "lsst.prompt.numDiaSourcesAll" where instrument = {instrument}"""  # where day_obs = {day_obs} '''
print(query)
df = await client.influx_client.query(query)
df

In [ ]:
df.dtypes

In [ ]:
for filt in plot_filter_colors.keys():
    wf = df["band"] == filt

    if np.sum(wf):
        plt.plot(
            df.loc[wf].index,
            df.loc[wf, "numDiaSourcesAll"],
            ".",
            color=plot_filter_colors[filt],
            label=filt,
        )

plt.legend()
plt.xlabel("Time")
plt.ylabel("Number of DiaSources per detector")

since prompt doesn't have useful data at the moment, query the other metrics to learn how

In [ ]:
client = EfdClient("usdfdev_efd", db_name="lsst.verify")

In [ ]:
await client.get_topics()

In [ ]:
query = """SELECT * FROM "lsst.verify.ap.totalUnassociatedDiaObjects" """
df = await client.influx_client.query(query)
df

In [ ]:
plt.plot(df.index, df.totalUnassociatedDiaObjects, ".")